In [1]:
import hashlib as hl                    # sha1
import secrets as rnd                   # randbelow, choice
import random as rnd2
import os, sys                          # output and logging
import numpy as np, scipy.stats as sp   # stats calculations
import matplotlib.pyplot as plt         # plotting

n = 32
n_bytes = n // 8
pad_w = 128 - n


In [2]:
# Function to generate <bits>-bit value
def generate(bits=256):
    return rnd.token_bytes(bits // 8)

def bytearr_to_bitstr(bytestr):
    return ''.join(format(b, '08b') for b in bytestr)

def h_n_bytes(byts):
    return hl.sha1(byts).digest()[-n_bytes:]

In [3]:
# def generate_bits(bits=256):
#     return rnd.randbits(bits).to_bytes(bits // 8, sys.byteorder)

def generate_bits(bits=256):
    return rnd2.randbytes(bits // 8)

a = generate_bits(256)
print(a)
len(a)

b'\x8b\xc1\xf2C\xfd\xcf>\x92\x1d\xa0\x12\xab\xff\xf0\x99\x82\xe3p_\xd6\xceV\x92F\x9f\xf6\x85\xda\xa5\xe1\xca\x14'


32

In [4]:
def gen_pre_table(K, L):
    r = generate_bits(pad_w)
    def R(x):
        return r + x

    X = {}

    for _ in range(K):
        x0 = generate_bits(n) 
        value = x0
        for _ in range(L):
            value = h_n_bytes(R(value))
        X[value] = x0
    
    return [X, r]

def find_preimage(L, p_table, r, hash_val):
    def R(x):
        return r + x    

    y = hash_val

    j_found = -1
    for j in range(L):
        # Check all i for xiL == y
        if y in p_table:
            j_found = j
            break

        # Next itr
        y = h_n_bytes(R(y))
    
    if j_found != -1:
        x = p_table[y]
        for _ in range(L - j_found):
            x = h_n_bytes(R(x))
        return [R(x), True]

    # Error
    return [None, False]
    

In [2]:
def foo(a, b):
    print(a, b)

foo([1, 2])

TypeError: foo() missing 1 required positional argument: 'b'

In [4]:
import numpy as np

N = 2**32
K = 2**20
L = 2**11


sum = 0
for i in range(1, K + 1):
    for j in range(L):
        sum += (1- i*L / N)**j / N

KeyboardInterrupt: 

In [ ]:
np.reduce()

In [3]:
print(sum)

0.0057360695389981255


In [5]:
K_test = 2**12
L_test = 2**12

value = generate(256)
h_val = h_n_bytes(value)

print(h_val)

X, r = gen_pre_table(K_test, L_test)
print("Table rdy!")
preimage, found = find_preimage(L_test, X, r, h_val)

if found:
    print(preimage)
    print(h_n_bytes(preimage))
else:
    print("No succ? :(")


b'x\xf3\x87\t'
Table rdy!
b'\xab\x8a\x85\xb3\xd6U\x1a\x9aB\x1b\xdbIrqz\xbb'
b'\x85\x0c\xb89'


In [6]:
from concurrent.futures import ProcessPoolExecutor, wait
from multiprocessing import Manager


def run_cpu_tasks_in_parallel(tasks, *parameters):
    print(*parameters)
    with ProcessPoolExecutor() as executor:
        running_tasks = [executor.submit(task, *parameters) for task in tasks]
        for running_task in running_tasks:
            running_task.result()
        # wait(running_tasks)

def R(r, x):
    return r + x

def append_value(data_pool, L, r):
    x0 = generate(n) 
    value = x0
    print(x0)
    for _ in range(L):
        value = h_n_bytes(R(r, value))
    print(value)
    data_pool[value] = x0

def gen_pre_table_parallel(K, L):
    r = generate(pad_w)

    X = {}
    with Manager() as mng:
        m_dict = mng.dict()

        run_cpu_tasks_in_parallel([append_value]*K, m_dict, L, r)
        print(m_dict)
        X = m_dict.copy()
        
    return [X, r]

In [7]:
h_n_bytes(b'1T\xb1?A\xacq\xfe\x96\xa8\xdcJ\x1c\x88\xf2\xc0')

b'\x88\x84\\ '

In [8]:
print(h_val)

X, r = gen_pre_table_parallel(K_test, L_test)
print(X)
print("Table rdy!")
preimage, found = find_preimage(L_test, X, r, h_val)

if found:
    print(preimage)
    print(h_n_bytes(preimage))
else:
    print("No succ? :(")

b'x\xf3\x87\t'
{} 4096 b'\x0c\r\xb0\xca\xd5$\xa6\xca\xe3\x10\xfaG'


BrokenProcessPool: A child process terminated abruptly, the process pool is not usable anymore

# Атака 1 (одна таблиця)

In [ ]:
K_arr = [2**20, 2**22, 2**24]
L_arr = [2**10, 2**11, 2**12]

# Атака 2 (K таблиць)

In [ ]:
K_arr = [2**10, 2**11, 2**12]
L_arr = [2**10, 2**11, 2**12]